In [1]:
%load_ext autoreload
%autoreload 2
import os
from huggingface_hub import login
from collections import defaultdict
from projects.wiki_experts.src.evolution.utils import get_svd_embedding

from mttl.models.modifiers.expert_containers.expert_library import LocalExpertLibrary,  HFExpertLibrary
from mttl.models.modifiers.expert_containers.library_transforms import SVDEmbeddingTransform, SVDEmbeddingTransformConfig
from huggingface_hub import login, HfApi
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity

/anaconda/envs/comp_3.9/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/anaconda/envs/comp_3.9/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda9SetDeviceEi'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


Registering modifier...lora
Registering modifier...poly
Registering modifier...per_token_poly
Registering modifier...skilled
Registering modifier...kv_adapter
Registering modifier...poly_kv_adapter
Registering modifier...prompt_tuning
Registering modifier...poly_prompt_tuning
Registering modifier...hard_prompt
Registering multi-expert selector...poly_router
Registering multi-expert selector...moe_rkhs_router
Registering multi-expert selector...zero_router
Registering multi-expert selector...zero_per_token_router
Registering multi-expert selector...poly_router_dir
Registering multi-expert selector...info_selector
Registering multi-expert selector...task_selector
Registering multi-expert selector...kv_task_selector
Registering multi-expert selector...kv_concat_selector
Registering multi-expert selector...kv_norm_selector
Registering multi-expert selector...kv_concat_norm_selector
Registering multi-expert selector...kv_task_norm_selector


## Create clsuters

In [5]:
hf_api_key = os.environ["HF_TOKEN"]
login(token=hf_api_key)
user = HfApi(token=hf_api_key).whoami()
hf_repo_id = "ostapeno/library-gptneo_1B_flan_2ep"
local_lib_location = f"/tmp/{hf_repo_id}"
if not os.path.exists(local_lib_location):
    os.makedirs(local_lib_location)
    expert_lib: LocalExpertLibrary = LocalExpertLibrary.create_from_remote(
        HFExpertLibrary(hf_repo_id), local_lib_location
    )
else:
    expert_lib: LocalExpertLibrary = LocalExpertLibrary(local_lib_location)

import json
task_set_path = "/home/v-oostapenko/dev/lucas_mttl/projects/wiki_experts/task_sets/flan_tasks.json"
flan256 = json.load(open(task_set_path))["flan256"]
        
for expert_name, expert in list(expert_lib.data.items()):
    if expert.expert_task_name not in flan256:
        expert_lib.remove_expert(expert_name)
assert len(expert_lib) == 256

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/v-oostapenko/.cache/huggingface/token
Login successful


In [6]:
for name, exp in expert_lib.data.items():
    # print task
    print(f"\"{exp.expert_task_name}\"")

"glue_sst2_2_0_0"
"dream_read_the_following_conversation_and_answer_the_question"
"race_middle_Read_the_article_and_answer_the_question_no_option_"
"adversarial_qa_droberta_generate_question"
"adversarial_qa_dbidaf_question_context_answer"
"app_reviews_convert_to_star_rating"
"race_high_Select_the_best_answer"
"super_glue_rte_1_0_2"
"true_case"
"wiqa_what_might_be_the_first_step_of_the_process"
"quail_description_context_question_answer_id"
"quail_context_question_description_text"
"stream_qed"
"huggingface_xsum"
"cos_e_v1_11_question_option_description_text"
"wiqa_what_is_the_final_step_of_the_following_process"
"ropes_background_new_situation_answer"
"wiki_qa_found_on_google"
"cot_esnli"
"social_i_qa_Show_choices_and_generate_answer"
"cot_gsm8k"
"app_reviews_categorize_rating_using_review"
"cot_sensemaking"
"trec_1_0_0"
"super_glue_wic_1_0_2"
"ropes_prompt_bottom_no_hint"
"quartz_answer_question_based_on"
"super_glue_record_1_0_2"
"yelp_polarity_reviews_0_2_0"
"race_middle_Is_this_th

In [7]:
def create_embeddings():
    svd_embedder = SVDEmbeddingTransform(
        SVDEmbeddingTransformConfig(sparsity_threshold=0.5),
        random_state=42,
    )
    embeddings, svd = svd_embedder.transform(expert_lib, upload_to_hf=True, force=True)
    del svd_embedder
    return embeddings, svd


embeds = expert_lib.get_auxiliary_data("embeddings")
if len(embeds) == 0:
    print("creating embeddings")
    _, svd = create_embeddings()

# module to embedding
module2embed = {}
for n, m in expert_lib.items():
    module2embed[n] = get_svd_embedding(expert_lib, n)

In [9]:
# Extract the embeddings as a numpy array
embeddings = np.array(list(module2embed.values()))
cosine_sim_matrix = cosine_similarity(embeddings, embeddings)
K = 10
kmeans = KMeans(n_clusters=K, init="k-means++", n_init=10, random_state=42)
kmeans.fit(cosine_sim_matrix)
cluster_labels = kmeans.labels_

In [12]:
clusters = defaultdict(list)
# Print the cluster labels for each embedding
for key, label in zip(module2embed.keys(), cluster_labels):
    clusters[label].append(key)

for c, l in clusters.items():
    print(f"Cluster {c} has {len(l)} elements")
    print(f"c{c}o{K}_2e = {l}")

Cluster 4 has 45 elements
c4o10_2e = ['glue_sst2_2_0_0', 'dream_read_the_following_conversation_and_answer_the_question', 'race_middle_Read_the_article_and_answer_the_question_no_option_', 'race_high_Select_the_best_answer', 'true_case', 'quail_description_context_question_answer_id', 'quail_context_question_description_text', 'stream_qed', 'cot_esnli', 'quoref_Context_Contains_Answer', 'race_high_Read_the_article_and_answer_the_question_no_option_', 'duorc_ParaphraseRC_movie_director', 'quail_context_description_question_answer_id', 'ag_news_subset_1_0_0', 'wiki_hop_original_generate_object', 'race_high_Taking_a_test', 'cos_e_v1_11_question_description_option_text', 'kilt_tasks_hotpotqa_combining_facts', 'race_middle_Select_the_best_answer', 'kilt_tasks_hotpotqa_straighforward_qa', 'quail_context_question_description_answer_id', 'quac_1_0_0', 'wiki_hop_original_explain_relation', 'quail_context_question_answer_description_text', 'race_high_Select_the_best_answer_generate_span_', 'race

Bad pipe message: %s [b'\x03q\x82\x93\x863vb\xeeZ\xdda\xf6\xe7\xc5\xfb\xd5\x92 \x1d\n\xafOv\x833\x93\x8e\xc9d\x92;\xbc\x1fz\xa5v\x801\xdcx\xbd\xf0p\xd6,[\x072D\xa6\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06\x01\x00+\x00\x03\x02\x03\x04\x00-\x00']
Bad pipe message: %s [b':yq[\x1b\xf5M\x80\xc6\x1eB=\x03~X\x91\xcc\x1f \x7f\x14$B\xed\xe3\n\x11o\xde\x83G\xea\xda1\x95T\xa8@@f\x86\x11b\x12x\x11f\x9fo\x99\x01\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x0

### Rand score

In [14]:
from sklearn.metrics.cluster import adjusted_rand_score

In [29]:
adjusted_rand_score(cluster_labels, cluster_labels_direct)

0.47611902409439244

# Save svd file

In [ ]:
hf_api_key = os.environ["HF_TOKEN"]
login(token=hf_api_key)
user = HfApi(token=hf_api_key).whoami()
hf_repo_id = "ostapeno/library-phi_2-v3-10-flan-clusters"
local_lib_location = f"/tmp/{hf_repo_id}"
if not os.path.exists(local_lib_location):
    os.makedirs(local_lib_location)
    expert_lib: LocalExpertLibrary = LocalExpertLibrary.create_from_remote(
        HFExpertLibrary(hf_repo_id), local_lib_location
    )
else:
    expert_lib: LocalExpertLibrary = LocalExpertLibrary(local_lib_location)

In [ ]:
_, svd = create_embeddings()

In [ ]:
# upload embeddings
remote_lib = HFExpertLibrary.from_local(
    expert_lib,
    hf_repo_id,
    force=True,
    upload_aux_data=True,
)

In [ ]:
import pickle
# To save
pickle.dump(svd, open("svd.pkl", "wb"))

### PLayground